In [277]:
from selenium import webdriver
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [590]:
def enterCity(src,dest,dt):
    global source
    global destination
    global src_click
    global dest_click
    global dat
    global cal_click
    global btn_click
    
   
    try:
        #getting the web element for source city
        source=driver.find_element_by_xpath("//input[@class='field bw-1 bs-solid w-100p p-2 box-border br-4 fs-2 c-neutral-900 h-8 bc-neutral-100 c-neutral-900 focus:bc-secondary-500 flex flex-middle flex-between t-all fs-2 focus:bc-secondary-500 bg-transparent bc-neutral-100 pr-2 pl-3 pt-2 pb-2 ba br-4 h-8 null']")
    except NoSuchElementException:
        print("Could not find Source City")
    #filling the input city                                         
    #time.sleep(1)
    source.clear()
    time.sleep(1)

    source.send_keys(src)
    time.sleep(5)

    try:
        #filling correct name from list as  source city
        src_click=driver.find_element_by_xpath("//li[@class='ls-reset br-4 w-100p px-2 dropdown__item c-neutral-900 fs-3 hover:bg-secondary-500 hover:c-white c-pointer']")
    except NoSuchElementException:
        print("Could Not fill source city")

    src_click.click()
    
    try:
        #getting the web element for destination city
        destination=driver.find_element_by_xpath("//input[@class='field bw-1 bs-solid w-100p p-2 box-border br-4 fs-2 c-neutral-900 h-8 bc-neutral-100 c-neutral-900 focus:bc-secondary-500 flex flex-middle flex-between t-all fs-2 focus:bc-secondary-500 bg-transparent bc-neutral-100 pr-2 pl-3 pt-2 pb-2 ba br-4 h-8']")
    except NoSuchElementException:
        print("Could not find destination city")
    #time.sleep(1)
    destination.clear()
    #time.sleep(1)
    destination.send_keys(dest)
    time.sleep(5)

    try:
        #filling correct name from list as  destination city
        dest_click=driver.find_element_by_xpath("//li[@class='ls-reset br-4 w-100p px-2 dropdown__item c-neutral-900 fs-3 hover:bg-secondary-500 hover:c-white c-pointer']")
    except NoSuchElementException:
        print("Could not fill destination")

    dest_click.click()

    try:
        #click the calender for journey date selection
        cal_click=driver.find_element_by_xpath("//div[@class='flex flex-middle flex-between p-relative homeCalender']/button")
    except NoSuchElementException:
        print("Could not click calender")

    cal_click.click()

    time.sleep(2)

    try:
    #selecting the journey date
        dat=driver.find_elements_by_xpath("//div[@class='DayPicker-Month'][1]//div[@class='p-1 day-gridContent flex flex-middle flex-column flex-center flex-top']")
    except NoSuchElementException:
        print("Could not find the specified date")


    for i in dat:
        if (i.text == dt):
            i.click()
            break
    time.sleep(1.5)   

    #clicking the search button
    btn_click=driver.find_element_by_xpath("//button[@class='px-7 bg-primary-500 hover:bg-primary-600 c-white bc-transparent c-pointer w-100p py-2 px-5 h-10 fs-4 fw-600 t-all button bs-solid tp-color td-500 bw-1 br-4 lh-solid box-border']")
    btn_click.click()


In [591]:
def collectData(src,dest):
    #scrolling the page till end
    html = driver.find_element_by_tag_name('html')
    for i in range(15):
        html.send_keys(Keys.PAGE_DOWN)  
        html.send_keys(Keys.PAGE_DOWN) 
        html.send_keys(Keys.PAGE_DOWN) 
        time.sleep(5)

    #Scraping the air line name
    airline=[]
    dep_city=[]
    arr_city=[]
    collection=driver.find_elements_by_xpath("//img[@class='br-2 o-hidden mx-1']")
    for i in collection:
        try:
            airline.append(i.get_attribute('alt'))
            dep_city.append(src)
            arr_city.append(dest)
        except NoSuchElementException:
            airline.append('-')


    #Scraping the date
    dep_date=[]
    dt=driver.find_element_by_xpath("//div[@class='c-pointer flex flex-column flex-middle flex-1 pb-2 bb bc-secondary-500 bw-2']/p")
    date=dt.text

    #Scraping depurture time
    departure=[]
    dep=driver.find_elements_by_xpath("//div[@class='ms-grid-column-1 ms-grid-row-1']/p")
    for i in dep:
        try:
            departure.append(i.text)
        except NoSuchElementException:
            departure.append('-')

    for i in range(len(departure)):
        dep_date.append(date)

    #Scraping arriva time
    arrival=[]
    arr=driver.find_elements_by_xpath("//div[@class='ms-grid-column-2 ms-grid-row-1']/p")
    for i in arr:
        try:
            arrival.append(i.text[:5])
        except NoSuchElementException:
            arrival.append('-')


    #Sraping the flight duration

    duration=[]
    dur=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p")
    for i in dur:
        try:
            duration.append(i.text.replace('h',':').replace('m','').replace(' ',''))
        except NoSuchElementException:
            duration.append('-')

    #Scraping the number of stops

    stops=[]
    stp=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
    for i in stp:
        try:
            stops.append(i.text.replace('h',':').replace('m','').replace(' ',''))
        except NoSuchElementException:
            stops.append('-')

    #scraping the ticket price

    price=[]
    prc=driver.find_elements_by_xpath("//div[@class='flex flex-column pl-2 flex-bottom']/p")
    for i in prc:
        try:
            price.append(i.text.replace('₹','').replace(',',''))
        except NoSuchElementException:
            price.append('-')

    dict1={"Airline":airline,'Dept_City':dep_city,'Arr_City':arr_city,'Departure':departure,'Dep_date':dep_date,'Arrival':arrival,"Duration":duration,'Stops':stops,'Price':price}
    frame=pd.DataFrame(dict1)


    return frame
                        

In [563]:
df = pd.DataFrame(columns = ['Airline','Dept_City','Arr_City', 'Departure', 'Dep_date','Arrival','Duration','Stops','Price'])


In [564]:
df.head()

,Airline,Dept_City,Arr_City,Departure,Dep_date,Arrival,Duration,Stops,Price


In [593]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
    #driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Mumbai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [594]:
df.head()

,Airline,Dept_City,Arr_City,Departure,Dep_date,Arrival,Duration,Stops,Price
0,Air Asia,Kolkata,Mumbai,20:00,"Fri, 25 Feb",06:35,10:35,1stop,7957
1,SpiceJet,Kolkata,Mumbai,09:35,"Fri, 25 Feb",12:20,2:45,non-stop,7958
2,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",00:45,2:50,non-stop,7958
3,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",08:40,10:45,1stop,7958
4,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",09:35,11:40,1stop,7958


In [598]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [608]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Chennai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [609]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Bangalore'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [610]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [611]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Mumbai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [612]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Mumbai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [615]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [616]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Bangalore'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)

driver.close()

In [618]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Goa'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [619]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Mumbai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [621]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [626]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Chennai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [627]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [628]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Bangalore'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [629]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)

time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Chennai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [631]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [635]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [637]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Ahmedabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [638]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [639]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Bangalore'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [640]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Bhubaneswar'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [645]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [647]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Jaipur'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [648]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [649]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [650]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Bhubaneswar'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [653]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Bhubaneswar'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

,Airline,Dept_City,Arr_City,Departure,Dep_date,Arrival,Duration,Stops,Price
34,Vistara,Bangalore,Bhubaneswar,08:00,"Fri, 25 Feb",12:25,28:25,1stop,23659
35,Vistara,Bangalore,Bhubaneswar,19:40,"Fri, 25 Feb",19:55,24:15,1stop,24552
36,Air India,Bangalore,Bhubaneswar,19:40,"Fri, 25 Feb",14:25,18:45,1stop,25234
37,Vistara,Bangalore,Bhubaneswar,21:10,"Fri, 25 Feb",09:05,11:55,1stop,25444
38,IndiGo,Bangalore,Bhubaneswar,07:00,"Fri, 25 Feb",12:05,5:5,1stop,28488


In [655]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [656]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Jaipur'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [657]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [658]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [666]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Ahmedabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()          

In [667]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Bhubaneswar'
dt='25'

enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [671]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Goa'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [672]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Jaipur'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [674]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [678]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [679]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Ahmedabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [680]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [681]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Goa'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [688]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [690]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Kolkata'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [691]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Mumbai'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [692]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Hyderabad'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [698]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Jaipur'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [699]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='New Delhi'
dt='25'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [700]:
def enterCity(src,dest,dt):
    global source
    global destination
    global src_click
    global dest_click
    global dat
    global cal_click
    global btn_click
    
   
    try:
        #getting the web element for source city
        source=driver.find_element_by_xpath("//input[@class='field bw-1 bs-solid w-100p p-2 box-border br-4 fs-2 c-neutral-900 h-8 bc-neutral-100 c-neutral-900 focus:bc-secondary-500 flex flex-middle flex-between t-all fs-2 focus:bc-secondary-500 bg-transparent bc-neutral-100 pr-2 pl-3 pt-2 pb-2 ba br-4 h-8 null']")
    except NoSuchElementException:
        print("Could not find Source City")
    #filling the input city                                         
    #time.sleep(1)
    source.clear()
    time.sleep(1)

    source.send_keys(src)
    time.sleep(5)

    try:
        #filling correct name from list as  source city
        src_click=driver.find_element_by_xpath("//li[@class='ls-reset br-4 w-100p px-2 dropdown__item c-neutral-900 fs-3 hover:bg-secondary-500 hover:c-white c-pointer']")
    except NoSuchElementException:
        print("Could Not fill source city")

    src_click.click()
    
    try:
        #getting the web element for destination city
        destination=driver.find_element_by_xpath("//input[@class='field bw-1 bs-solid w-100p p-2 box-border br-4 fs-2 c-neutral-900 h-8 bc-neutral-100 c-neutral-900 focus:bc-secondary-500 flex flex-middle flex-between t-all fs-2 focus:bc-secondary-500 bg-transparent bc-neutral-100 pr-2 pl-3 pt-2 pb-2 ba br-4 h-8']")
    except NoSuchElementException:
        print("Could not find destination city")
    #time.sleep(1)
    destination.clear()
    #time.sleep(1)
    destination.send_keys(dest)
    time.sleep(5)

    try:
        #filling correct name from list as  destination city
        dest_click=driver.find_element_by_xpath("//li[@class='ls-reset br-4 w-100p px-2 dropdown__item c-neutral-900 fs-3 hover:bg-secondary-500 hover:c-white c-pointer']")
    except NoSuchElementException:
        print("Could not fill destination")

    dest_click.click()

    try:
        #click the calender for journey date selection
        cal_click=driver.find_element_by_xpath("//div[@class='flex flex-middle flex-between p-relative homeCalender']/button")
    except NoSuchElementException:
        print("Could not click calender")

    cal_click.click()

    time.sleep(2)

    try:
    #selecting the journey date
        dat=driver.find_elements_by_xpath("//div[@class='DayPicker-Month'][2]//div[@class='p-1 day-gridContent flex flex-middle flex-column flex-center flex-top']")
    except NoSuchElementException:
        print("Could not find the specified date")


    for i in dat:
        if (i.text == dt):
            i.click()
            break
    time.sleep(1.5)   

    #clicking the search button
    btn_click=driver.find_element_by_xpath("//button[@class='px-7 bg-primary-500 hover:bg-primary-600 c-white bc-transparent c-pointer w-100p py-2 px-5 h-10 fs-4 fw-600 t-all button bs-solid tp-color td-500 bw-1 br-4 lh-solid box-border']")
    btn_click.click()


In [701]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
    #driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Mumbai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [703]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [704]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Chennai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [707]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Bangalore'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [708]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [709]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Mumbai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [710]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Kolkata'
dest='Mumbai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [711]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [719]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Bangalore'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [721]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Goa'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [722]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Mumbai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [723]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [724]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='New Delhi'
dest='Chennai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [726]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [728]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Bangalore'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [729]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)

time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Chennai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [730]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [731]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [732]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Mumbai'
dest='Ahmedabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [733]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [737]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Bangalore'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [738]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Bhubaneswar'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [739]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [748]:
#--------------------------------------------------------
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='Jaipur'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [749]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Chennai'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()



In [751]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [753]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Bhubaneswar'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [754]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Bhubaneswar'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [755]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [756]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Jaipur'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [759]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
#calling function for source and destination city with departure datre 
src='Bangalore'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [762]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [763]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Jaipur'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()   

In [764]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Bhubaneswar'
dt='5'

enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [765]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bangalore'
dest='Goa'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [766]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='Jaipur'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [767]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Hyderabad'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [768]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [769]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Ahmedabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [770]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [772]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='Goa'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [773]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Bhubaneswar'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [775]:

driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Kolkata'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [777]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Mumbai'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()


In [778]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Hyderabad'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [779]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='Jaipur'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [780]:
driver=webdriver.Firefox(executable_path="C:\\Users\\User\\geckodriver.exe")
#driver.maximize_window()
url='https://www.cleartrip.com/flights'
driver.get(url)
time.sleep(10)
# calling function for source and destination city with departure datre 
src='Ahmedabad'
dest='New Delhi'
dt='5'
enterCity(src,dest,dt)
data=pd.DataFrame(collectData(src,dest))
df=df.append(data)
driver.close()

In [781]:
df.shape

(6161, 9)

In [782]:
df.isnull().sum()

Airline      0
Dept_City    0
Arr_City     0
Departure    0
Dep_date     0
Arrival      0
Duration     0
Stops        0
Price        0
dtype: int64

In [784]:
df.to_csv('scraped_flight_price.csv')

In [785]:
df.head(10)

,Airline,Dept_City,Arr_City,Departure,Dep_date,Arrival,Duration,Stops,Price
0,Air Asia,Kolkata,Mumbai,20:00,"Fri, 25 Feb",06:35,10:35,1stop,7957
1,SpiceJet,Kolkata,Mumbai,09:35,"Fri, 25 Feb",12:20,2:45,non-stop,7958
2,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",00:45,2:50,non-stop,7958
3,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",08:40,10:45,1stop,7958
4,SpiceJet,Kolkata,Mumbai,21:55,"Fri, 25 Feb",09:35,11:40,1stop,7958
5,Air India,Kolkata,Mumbai,09:40,"Fri, 25 Feb",12:25,2:45,non-stop,7959
6,GO FIRST,Kolkata,Mumbai,04:55,"Fri, 25 Feb",07:45,2:50,non-stop,7959
7,IndiGo,Kolkata,Mumbai,07:30,"Fri, 25 Feb",10:20,2:50,non-stop,7959
8,IndiGo,Kolkata,Mumbai,10:10,"Fri, 25 Feb",13:05,2:55,non-stop,7959
9,IndiGo,Kolkata,Mumbai,08:55,"Fri, 25 Feb",12:05,3:10,non-stop,7959
